In [1]:
from mlxtend.evaluate import bias_variance_decomp

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
import math

In [2]:
#Reading in 2016 data we cleaned
df2016 = pd.read_csv('data/exported_data.csv')


In [3]:
df2016

,Democrats 2016,Republicans 2016,Democrats 2012,Republicans 2012,State,County,Fips,Less Than High School Diploma,At Least High School Diploma,White (Not Latino) Population,Asian American Population,African American Population,Construction.extraction.maintenance.and.repair.occupations,Average Unemployment % 2016,"Poverty Percent, All Ages",realGDP2016,At Least Bachelors's Degree,Graduate Degree,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per
0,33.264033,62.176022,45.084746,53.587571,Michigan,"Presque Isle County, Michigan",26141,14.7,85.3,97.05,0.20,0.30,13.85,9.6,14.7,393475,14.3,4.7,0.070504,0.037034,0.033471,0.193137,0.097412,0.095725,0.376055,0.185824,0.190231,0.360304,0.178417,0.181886
1,41.088014,53.565475,50.177448,48.576131,Michigan,"Calhoun County, Michigan",26025,12.9,87.1,80.65,1.60,10.45,8.35,4.8,16.8,5594998,18.8,6.5,0.118989,0.061094,0.057895,0.308407,0.151431,0.156975,0.352965,0.173308,0.179657,0.219640,0.096424,0.123215
2,28.744694,66.858702,40.316090,58.523357,Michigan,"Branch County, Michigan",26023,13.4,86.6,90.35,0.60,3.05,8.05,4.8,16.8,1245185,14.1,4.8,0.101433,0.054194,0.047239,0.309689,0.170651,0.139038,0.362648,0.189468,0.173180,0.226230,0.103059,0.123171
3,36.855246,56.491599,51.061492,47.394519,Michigan,"Shiawassee County, Michigan",26155,10.7,89.3,95.45,0.30,0.45,12.05,5.3,12.0,1602095,14.9,5.6,0.110296,0.057764,0.052533,0.289581,0.146615,0.142966,0.379568,0.186526,0.193042,0.220555,0.099090,0.121465
4,34.247700,61.432326,43.482538,55.697655,Michigan,"Mackinac County, Michigan",26097,10.9,89.1,76.80,0.60,1.65,13.85,9.9,13.7,616120,20.1,7.6,0.078798,0.043072,0.035726,0.220924,0.114079,0.106845,0.384307,0.193322,0.190985,0.315971,0.155036,0.160935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059,24.798206,69.058296,38.451209,58.460161,Minnesota,"Lake of the Woods County, Minnesota",27077,10.4,89.6,95.55,0.45,0.25,9.90,4.8,10.9,167599,18.6,3.8,0.083898,0.044208,0.039690,0.225234,0.112294,0.112940,0.413682,0.218135,0.195547,0.277186,0.139400,0.137786
1060,39.551402,52.336449,48.513800,48.938429,Minnesota,"Stevens County, Minnesota",27149,10.3,89.7,92.30,1.05,1.20,5.90,3.0,11.1,618864,23.5,8.4,0.261536,0.126063,0.135473,0.263857,0.143207,0.120650,0.263083,0.131993,0.131090,0.211524,0.092807,0.118716
1061,26.105645,67.456300,36.962071,60.694748,Minnesota,"Martin County, Minnesota",27091,13.1,86.9,95.00,0.75,0.30,8.55,4.1,12.1,1014414,17.5,4.9,0.094755,0.048661,0.046094,0.259678,0.132888,0.126789,0.358477,0.180394,0.178083,0.287090,0.129293,0.157797
1062,35.231714,58.626466,51.055766,46.616134,Minnesota,"Traverse County, Minnesota",27155,11.9,88.1,93.50,0.15,0.15,12.55,3.9,12.7,204156,14.7,2.7,0.094589,0.049943,0.044646,0.235717,0.122588,0.113129,0.354900,0.181990,0.172910,0.314794,0.144154,0.170639


In [4]:
# Reading in 2020 data we collected 

# Population broken down by age
state13 = pd.read_csv('data/cc-est2019-agesex-13.csv')
state19 = pd.read_csv('data/cc-est2019-agesex-19.csv')
state12 = pd.read_csv('data/cc-est2019-agesex-12.csv')
state27 = pd.read_csv('data/cc-est2019-agesex-27.csv')
state32 = pd.read_csv('data/cc-est2019-agesex-32.csv')
state37 = pd.read_csv('data/cc-est2019-agesex-37.csv')
state39 = pd.read_csv('data/cc-est2019-agesex-39.csv')
state42 = pd.read_csv('data/cc-est2019-agesex-42.csv')
state48 = pd.read_csv('data/cc-est2019-agesex-48.csv')
state55 = pd.read_csv('data/cc-est2019-agesex-55.csv')
state26 = pd.read_csv('data/cc-est2019-agesex-26.csv')
state4 = pd.read_csv('data/cc-est2019-agesex-04.csv')
df_states = state13.append(state19).append(state12).append(state27).append(state32).append(state37).append(state39).append(state42).append(state48).append(state55).append(state26).append(state4)

df_states = df_states[df_states['YEAR'] == 12] #12 represents 2019 (as close to 2020 as we could find)




In [5]:
df_states.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,UNDER5_MALE,UNDER5_FEM,AGE513_TOT,AGE513_MALE,AGE513_FEM,AGE1417_TOT,AGE1417_MALE,AGE1417_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE16PLUS_TOT,AGE16PLUS_MALE,AGE16PLUS_FEM,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1544_TOT,AGE1544_MALE,AGE1544_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,AGE04_TOT,AGE04_MALE,AGE04_FEM,AGE59_TOT,AGE59_MALE,AGE59_FEM,AGE1014_TOT,AGE1014_MALE,AGE1014_FEM,AGE1519_TOT,AGE1519_MALE,AGE1519_FEM,AGE2024_TOT,AGE2024_MALE,AGE2024_FEM,AGE2529_TOT,AGE2529_MALE,AGE2529_FEM,AGE3034_TOT,AGE3034_MALE,AGE3034_FEM,AGE3539_TOT,AGE3539_MALE,AGE3539_FEM,AGE4044_TOT,AGE4044_MALE,AGE4044_FEM,AGE4549_TOT,AGE4549_MALE,AGE4549_FEM,AGE5054_TOT,AGE5054_MALE,AGE5054_FEM,AGE5559_TOT,AGE5559_MALE,AGE5559_FEM,AGE6064_TOT,AGE6064_MALE,AGE6064_FEM,AGE6569_TOT,AGE6569_MALE,AGE6569_FEM,AGE7074_TOT,AGE7074_MALE,AGE7074_FEM,AGE7579_TOT,AGE7579_MALE,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
11,50,13,1,Georgia,Appling County,12,18386,9157,9229,1177,608,569,2359,1186,1173,977,492,485,1520,848,672,14347,7115,7232,13873,6871,7002,6592,3430,3162,4366,2221,2145,4730,2324,2406,3257,1478,1779,1177,608,569,1306,655,651,1324,662,662,1124,598,526,1102,611,491,1127,620,507,1017,491,526,1090,555,535,1132,555,577,1111,537,574,1128,578,550,1241,624,617,1250,585,665,1006,482,524,929,465,464,605,272,333,398,150,248,319,109,210,39.7,37.9,41.2
23,50,13,3,Georgia,Atkinson County,12,8165,4146,4019,573,300,273,1069,550,519,498,259,239,751,373,378,6280,3175,3105,6025,3037,2988,3164,1620,1544,2035,1048,987,2056,1043,1013,1183,573,610,573,300,273,595,307,288,594,303,291,584,300,284,545,272,273,547,280,267,498,269,229,462,228,234,528,271,257,512,265,247,534,297,237,505,241,264,505,240,265,392,192,200,325,161,164,228,105,123,148,71,77,90,44,46,36.6,35.9,37.4
35,50,13,5,Georgia,Bacon County,12,11164,5455,5709,766,381,385,1424,719,705,643,313,330,940,500,440,8651,4204,4447,8331,4042,4289,4219,2125,2094,2788,1387,1401,2725,1340,1385,1878,815,1063,766,381,385,774,390,384,802,404,398,753,381,372,678,357,321,684,348,336,679,346,333,631,290,341,794,403,391,717,372,345,706,350,356,629,313,316,673,305,368,671,318,353,479,228,251,323,131,192,243,90,153,162,48,114,38.6,37.2,39.8
47,50,13,7,Georgia,Baker County,12,3038,1494,1544,175,97,78,281,139,142,135,58,77,227,109,118,2513,1228,1285,2447,1200,1247,947,469,478,618,317,301,883,436,447,719,338,381,175,97,78,138,75,63,176,79,97,162,74,88,167,78,89,169,93,76,156,80,76,151,67,84,142,77,65,196,98,98,190,90,100,251,120,131,246,128,118,238,118,120,178,80,98,131,63,68,69,29,40,103,48,55,47.3,46.8,47.9
59,50,13,9,Georgia,Baldwin County,12,44890,22737,22153,2113,1116,997,4262,2158,2104,1913,991,922,8380,4086,4294,37528,18950,18578,36602,18472,18130,19737,10098,9639,9947,5283,4664,10877,5662,5215,7398,3441,3957,2113,1116,997,2278,1130,1148,2487,1290,1197,4360,2049,2311,5430,2766,2664,2846,1517,1329,2420,1286,1134,2452,1284,1168,2229,1196,1033,2458,1320,1138,2631,1397,1234,2925,1490,1435,2863,1455,1408,2452,1203,1249,2008,916,1092,1333,617,716,863,384,479,742,321,421,36.1,35.9,36.3


In [6]:
df_states

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,UNDER5_MALE,UNDER5_FEM,AGE513_TOT,AGE513_MALE,AGE513_FEM,AGE1417_TOT,AGE1417_MALE,AGE1417_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE16PLUS_TOT,AGE16PLUS_MALE,AGE16PLUS_FEM,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1544_TOT,AGE1544_MALE,AGE1544_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,AGE04_TOT,AGE04_MALE,AGE04_FEM,AGE59_TOT,AGE59_MALE,AGE59_FEM,AGE1014_TOT,AGE1014_MALE,AGE1014_FEM,AGE1519_TOT,AGE1519_MALE,AGE1519_FEM,AGE2024_TOT,AGE2024_MALE,AGE2024_FEM,AGE2529_TOT,AGE2529_MALE,AGE2529_FEM,AGE3034_TOT,AGE3034_MALE,AGE3034_FEM,AGE3539_TOT,AGE3539_MALE,AGE3539_FEM,AGE4044_TOT,AGE4044_MALE,AGE4044_FEM,AGE4549_TOT,AGE4549_MALE,AGE4549_FEM,AGE5054_TOT,AGE5054_MALE,AGE5054_FEM,AGE5559_TOT,AGE5559_MALE,AGE5559_FEM,AGE6064_TOT,AGE6064_MALE,AGE6064_FEM,AGE6569_TOT,AGE6569_MALE,AGE6569_FEM,AGE7074_TOT,AGE7074_MALE,AGE7074_FEM,AGE7579_TOT,AGE7579_MALE,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
11,50,13,1,Georgia,Appling County,12,18386,9157,9229,1177,608,569,2359,1186,1173,977,492,485,1520,848,672,14347,7115,7232,13873,6871,7002,6592,3430,3162,4366,2221,2145,4730,2324,2406,3257,1478,1779,1177,608,569,1306,655,651,1324,662,662,1124,598,526,1102,611,491,1127,620,507,1017,491,526,1090,555,535,1132,555,577,1111,537,574,1128,578,550,1241,624,617,1250,585,665,1006,482,524,929,465,464,605,272,333,398,150,248,319,109,210,39.7,37.9,41.2
23,50,13,3,Georgia,Atkinson County,12,8165,4146,4019,573,300,273,1069,550,519,498,259,239,751,373,378,6280,3175,3105,6025,3037,2988,3164,1620,1544,2035,1048,987,2056,1043,1013,1183,573,610,573,300,273,595,307,288,594,303,291,584,300,284,545,272,273,547,280,267,498,269,229,462,228,234,528,271,257,512,265,247,534,297,237,505,241,264,505,240,265,392,192,200,325,161,164,228,105,123,148,71,77,90,44,46,36.6,35.9,37.4
35,50,13,5,Georgia,Bacon County,12,11164,5455,5709,766,381,385,1424,719,705,643,313,330,940,500,440,8651,4204,4447,8331,4042,4289,4219,2125,2094,2788,1387,1401,2725,1340,1385,1878,815,1063,766,381,385,774,390,384,802,404,398,753,381,372,678,357,321,684,348,336,679,346,333,631,290,341,794,403,391,717,372,345,706,350,356,629,313,316,673,305,368,671,318,353,479,228,251,323,131,192,243,90,153,162,48,114,38.6,37.2,39.8
47,50,13,7,Georgia,Baker County,12,3038,1494,1544,175,97,78,281,139,142,135,58,77,227,109,118,2513,1228,1285,2447,1200,1247,947,469,478,618,317,301,883,436,447,719,338,381,175,97,78,138,75,63,176,79,97,162,74,88,167,78,89,169,93,76,156,80,76,151,67,84,142,77,65,196,98,98,190,90,100,251,120,131,246,128,118,238,118,120,178,80,98,131,63,68,69,29,40,103,48,55,47.3,46.8,47.9
59,50,13,9,Georgia,Baldwin County,12,44890,22737,22153,2113,1116,997,4262,2158,2104,1913,991,922,8380,4086,4294,37528,18950,18578,36602,18472,18130,19737,10098,9639,9947,5283,4664,10877,5662,5215,7398,3441,3957,2113,1116,997,2278,1130,1148,2487,1290,1197,4360,2049,2311,5430,2766,2664,2846,1517,1329,2420,1286,1134,2452,1284,1168,2229,1196,1033,2458,1320,1138,2631,1397,1234,2925,1490,1435,2863,1455,1408,2452,1203,1249,2008,916,1092,1333,617,716,863,384,479,742,321,421,36.1,35.9,36.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,50,4,19,Arizona,Pima County,12,1047279,515596,531683,57018,29143,27875,109409,55916,53493,48880,24971,23909,121421,61475,59946,856311,417927,438384,831972,405566,426406,414104,211581,202523,256021,131400,124621,241976,116100,125876,212554,96591,115963,57018,29143,27875,59650,30372,29278,61977,31809,30168,686

In [7]:
# Getting the columns we want
df_states = df_states[['STATE', 'COUNTY', 'STNAME', 'CTYNAME','AGE18PLUS_TOT','AGE18PLUS_MALE','AGE18PLUS_FEM', 'AGE1824_TOT', 'AGE1824_MALE','AGE1824_FEM', 
                       'AGE2544_TOT','AGE2544_MALE','AGE2544_FEM', 'AGE4564_TOT','AGE4564_MALE', 'AGE4564_FEM',
                      'AGE65PLUS_TOT', 'AGE65PLUS_MALE', 'AGE65PLUS_FEM']]

In [8]:
# Dividing by total value to get a percentage 
df_states['1824tot_per'] = df_states['AGE1824_TOT']/df_states['AGE18PLUS_TOT']
df_states['1824M_per'] = df_states['AGE1824_MALE']/df_states['AGE18PLUS_TOT']
df_states['1824F_per'] = df_states['AGE1824_FEM']/df_states['AGE18PLUS_TOT']
df_states['2544tot_per'] = df_states['AGE2544_TOT']/df_states['AGE18PLUS_TOT']
df_states['2544M_per'] = df_states['AGE2544_MALE']/df_states['AGE18PLUS_TOT']
df_states['2544F_per'] = df_states['AGE2544_FEM']/df_states['AGE18PLUS_TOT']
df_states['4564tot_per'] = df_states['AGE4564_TOT']/df_states['AGE18PLUS_TOT']
df_states['4564M_per'] = df_states['AGE4564_MALE']/df_states['AGE18PLUS_TOT']
df_states['4564F_per'] = df_states['AGE4564_FEM']/df_states['AGE18PLUS_TOT']
df_states['65tot_per'] = df_states['AGE65PLUS_TOT']/df_states['AGE18PLUS_TOT']
df_states['65M_per'] = df_states['AGE65PLUS_MALE']/df_states['AGE18PLUS_TOT']
df_states['65F_per'] = df_states['AGE65PLUS_FEM']/df_states['AGE18PLUS_TOT']

<ipython-input-8-195acd64b013>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_states['1824tot_per'] = df_states['AGE1824_TOT']/df_states['AGE18PLUS_TOT']
<ipython-input-8-195acd64b013>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_states['1824M_per'] = df_states['AGE1824_MALE']/df_states['AGE18PLUS_TOT']
<ipython-input-8-195acd64b013>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [9]:
df_states.head()

,STATE,COUNTY,STNAME,CTYNAME,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per
11,13,1,Georgia,Appling County,13873,6871,7002,1520,848,672,4366,2221,2145,4730,2324,2406,3257,1478,1779,0.109565,0.061126,0.048439,0.314712,0.160095,0.154617,0.340950,0.167520,0.173430,0.234773,0.106538,0.128235
23,13,3,Georgia,Atkinson County,6025,3037,2988,751,373,378,2035,1048,987,2056,1043,1013,1183,573,610,0.124647,0.061909,0.062739,0.337759,0.173942,0.163817,0.341245,0.173112,0.168133,0.196349,0.095104,0.101245
35,13,5,Georgia,Bacon County,8331,4042,4289,940,500,440,2788,1387,1401,2725,1340,1385,1878,815,1063,0.112832,0.060017,0.052815,0.334654,0.166487,0.168167,0.327092,0.160845,0.166247,0.225423,0.097827,0.127596
47,13,7,Georgia,Baker County,2447,1200,1247,227,109,118,618,317,301,883,436,447,719,338,381,0.092767,0.044544,0.048222,0.252554,0.129546,0.123008,0.360850,0.178177,0.182673,0.293829,0.138128,0.155701
59,13,9,Georgia,Baldwin County,36602,18472,18130,8380,4086,4294,9947,5283,4664,10877,5662,5215,7398,3441,3957,0.228949,0.111633,0.117316,0.271761,0.144336,0.127425,0.297170,0.154691,0.142479,0.202120,0.094011,0.108109


In [10]:
df_states = df_states.reset_index()

In [11]:
# 2019 American Community Survey data about education level
acs_edu = pd.read_csv('data/acs_data_education.csv', header = 1)

In [12]:
acs_edu

,id,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma","Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree","Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT

In [13]:
acs_edu.head()

,id,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma","Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree","Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT

In [14]:
# 2019 American Community Survey data about racial makeup
acs_race = pd.read_csv('data/acs_data_race.csv', header = 1)

In [15]:
acs_race

,id,Geographic Area Name,Estimate!!Total:,Margin of Error!!Total:,Estimate!!Total:!!White alone,Margin of Error!!Total:!!White alone,Estimate!!Total:!!Black or African American alone,Margin of Error!!Total:!!Black or African American alone,Estimate!!Total:!!American Indian and Alaska Native alone,Margin of Error!!Total:!!American Indian and Alaska Native alone,Estimate!!Total:!!Asian alone,Margin of Error!!Total:!!Asian alone,Estimate!!Total:!!Native Hawaiian and Other Pacific Islander alone,Margin of Error!!Total:!!Native Hawaiian and Other Pacific Islander alone,Estimate!!Total:!!Some other race alone,Margin of Error!!Total:!!Some other race alone,Estimate!!Total:!!Two or more races:,Margin of Error!!Total:!!Two or more races:,Estimate!!Total:!!Two or more races:!!Two races including Some other race,Margin of Error!!Total:!!Two or more races:!!Two races including Some other race,"Estimate!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races","Margin of Error!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races"
0,0500000US04001,"Apache County, Arizona",71887.0,*****,15109.0,899.0,721.0,434.0,53480.0,1032.0,160.0,168.0,15.0,27.0,1127.0,595.0,1275.0,743.0,637.0,509.0,638.0,514.0
1,0500000US04003,"Cochise County, Arizona",125922.0,*****,108177.0,1768.0,6163.0,1237.0,1006.0,911.0,2795.0,674.0,129.0,218.0,2141.0,1071.0,5511.0,1259.0,448.0,334.0,5063.0,1164.0
2,0500000US04005,"Coconino County, Arizona",143476.0,*****,91649.0,1736.0,1364.0,461.0,37187.0,1082.0,2730.0,481.0,419.0,440.0,3897.0,1334.0,6230.0,1766.0,253.0,305.0,5977.0,1750.0
3,0500000US04013,"Maricopa County, Arizona",4485414.0,*****,3547155.0,17870.0,266128.0,7213.0,93358.0,6179.0,192301.0,5275.0,10045.0,1308.0,213953.0,16250.0,162474.0,9681.0,31995.0,5459.0,130479.0,8908.0
4,0500000US04015,"Mohave County, Arizona",212181.0,*****,188756.0,3072.0,1496.0,845.0,4358.0,1173.0,1923.0,474.0,138.0,225.0,7608.0,3077.0,7902.0,1920.0,1169.0,605.0,6733.0,1827.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,0500000US55127,"Walworth County, Wisconsin",103868.0,*****,97838.0,1044.0,1416.0,568.0,359.0,353.0,409.0,310.0,0.0,153.0,1948.0,971.0,1898.0,658.0,317.0,244.0,1581.0,605.0
337,0500000US55131,"Washington County, Wisconsin",136034.0,*****,128012.0,751.0,2501.0,490.0,224.0,323.0,2643.0,506.0,57.0,104.0,1315.0,762.0,1282.0,798.0,395.0,434.0,887.0,645.0
338,0500000US55133,"Waukesha County, Wisconsin",404198.0,*****,370945.0,2169.0,6660.0,1419.0,912.0,363.0,15415.0,1092.0,257.0,256.0,3725.0,2262.0,6284.0,1847.0,696.0,506.0,5588.0,1773.0
339,0500000US55139,"Winnebago County, Wisconsin",171907.0,*****,155315.0,2212.0,4049.0,781.0,1084.0,343.0,4560.0,1278.0,986.0,1311.0,3186.0,2230.0,2727.0,998.0,51.0,81.0,2676.0,1016.0


In [16]:
acs_race.head()

,id,Geographic Area Name,Estimate!!Total:,Margin of Error!!Total:,Estimate!!Total:!!White alone,Margin of Error!!Total:!!White alone,Estimate!!Total:!!Black or African American alone,Margin of Error!!Total:!!Black or African American alone,Estimate!!Total:!!American Indian and Alaska Native alone,Margin of Error!!Total:!!American Indian and Alaska Native alone,Estimate!!Total:!!Asian alone,Margin of Error!!Total:!!Asian alone,Estimate!!Total:!!Native Hawaiian and Other Pacific Islander alone,Margin of Error!!Total:!!Native Hawaiian and Other Pacific Islander alone,Estimate!!Total:!!Some other race alone,Margin of Error!!Total:!!Some other race alone,Estimate!!Total:!!Two or more races:,Margin of Error!!Total:!!Two or more races:,Estimate!!Total:!!Two or more races:!!Two races including Some other race,Margin of Error!!Total:!!Two or more races:!!Two races including Some other race,"Estimate!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races","Margin of Error!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races"
0,0500000US04001,"Apache County, Arizona",71887.0,*****,15109.0,899.0,721.0,434.0,53480.0,1032.0,160.0,168.0,15.0,27.0,1127.0,595.0,1275.0,743.0,637.0,509.0,638.0,514.0
1,0500000US04003,"Cochise County, Arizona",125922.0,*****,108177.0,1768.0,6163.0,1237.0,1006.0,911.0,2795.0,674.0,129.0,218.0,2141.0,1071.0,5511.0,1259.0,448.0,334.0,5063.0,1164.0
2,0500000US04005,"Coconino County, Arizona",143476.0,*****,91649.0,1736.0,1364.0,461.0,37187.0,1082.0,2730.0,481.0,419.0,440.0,3897.0,1334.0,6230.0,1766.0,253.0,305.0,5977.0,1750.0
3,0500000US04013,"Maricopa County, Arizona",4485414.0,*****,3547155.0,17870.0,266128.0,7213.0,93358.0,6179.0,192301.0,5275.0,10045.0,1308.0,213953.0,16250.0,162474.0,9681.0,31995.0,5459.0,130479.0,8908.0
4,0500000US04015,"Mohave County, Arizona",212181.0,*****,188756.0,3072.0,1496.0,845.0,4358.0,1173.0,1923.0,474.0,138.0,225.0,7608.0,3077.0,7902.0,1920.0,1169.0,605.0,6733.0,1827.0


In [17]:
# Dropping columns we don't need in both of them

cols_edu = []
for i in range(3,56):
    if i%2!=0:
        cols_edu.append(i)
        
cols_edu_rest = list(range(56,770))

acs_edu = acs_edu.drop(acs_edu.columns[cols_edu_rest], axis = 1)
acs_edu = acs_edu.drop(acs_edu.columns[cols_edu], axis = 1)
acs_edu.head()

,id,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over!!Bachelor's degree or higher
0,0500000US04001,"Apache County, Arizona",6532,1054,2127,3187,164,46255,4149,4297,14470,13772,4095,2837,2635,37809,5472,9316,8025,860,7153,6682,801,18183,15457,2125,11603,7645,1686
1,0500000US04003,"Cochise County, Arizona",10486,2191,3103,5114,78,88433,4006,6043,23731,22329,10223,13332,8769,78384,22101,15467,13438,3639,14188,12235,3504,29839,26707,6641,28939,26004,8317
2,0500000US04005,"Coconino County, Arizona",30789,1290,8590,18963,1946,84682,1910,3606,17279,21288,7088,18637,14874,79166,33511,19997,19184,7735,15283,14223,6963,30816,29051,10833,18586,16708,7980
3,0500000US04013,"Maricopa County, Arizona",415899,58736,149802,155679,51682,3017076,157753,200414,671626,710797,264954,641198,370334,2658909,1011532,658682,592116,215671,583814,501914,203036,1078113,941895,357351,696467,622984,235474
4,0500000US04015,"Mohave County, Arizona",13365,2782,6209,3876,498,163211,4079,17157,55224,50138,14602,13958,8053,141975,22011,20109,16498,1469,19281,17378,3652,57994,51636,7972,65827,56463,8918


In [18]:
acs_edu['Less than High School'] = (acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate'] + 
    acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma'] + 
    acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade'])/(acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over'] + 
    acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years'])
acs_edu['At Least High School'] = (acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency)'] + 
    acs_edu["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency)"])/(acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years']+
                                                                                                                                     acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over'])

acs_edu["At Least Bachelor's Degree"] = (acs_edu["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher"]+
                                         acs_edu["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree"])/(acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years']+
                                                                                                                                     acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over'])
acs_edu['Graduate Degree'] = (acs_edu["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher"]+
                             acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree'])/(acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years']+
                                                                                                                                     acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over'])


acs_edu.head()

,id,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over!!Bachelor's degree or higher,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree
0,0500000US04001,"Apache County, Arizona",6532,1054,2127,3187,164,46255,4149,4297,14470,13772,4095,2837,2635,37809,5472,9316,8025,860,7153,6682,801,18183,15457,2125,11603,7645,1686,0.179969,0.314415,0.056851,0.053024
1,0500000US04003,"Cochise County, Arizona",10486,2191,3103,5114,78,88433,4006,6043,23731,22329,10223,13332,8769,78384,22101,15467,13438,3639,14188,12235,3504,29839,26707,6641,28939,26004,8317,0.123738,0.271272,0.135565,0.089437
2,0500000US04005,"Coconino County, Arizona",30789,1290,8590,18963,1946,84682,1910,3606,17279,21288,7088,18637,14874,79166,33511,19997,19184,7735,15283,14223,6963,30816,29051,10833,18586,16708,7980,0.058941,0.224030,0.178253,0.145664
3,0500000US04013,"Maricopa County, Arizona",415899,58736,149802,155679,51682,3017076,157753,200414,671626,710797,264954,641198,370334,2658909,1011532,658682,592116,215671,583814,501914,203036,1078113,941895,357351,696467,622984,235474,0.121441,0.239276,0.201831,0.122930
4,0500000US04015,"Mohave County, Arizona",13365,2782,6209,3876,498,163211,4079,17157,55224,50138,14602,13958,8053,141975,22011,20109,16498,1469,19281,17378,3652,57994,51636,7972,65827,56463,8918,0.136021,0.347913,0.081868,0.048427


In [19]:
cols_edu = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
acs_edu = acs_edu.drop(acs_edu.columns[cols_edu],axis = 1)
acs_edu.head()

,id,Geographic Area Name,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree
0,0500000US04001,"Apache County, Arizona",0.179969,0.314415,0.056851,0.053024
1,0500000US04003,"Cochise County, Arizona",0.123738,0.271272,0.135565,0.089437
2,0500000US04005,"Coconino County, Arizona",0.058941,0.224030,0.178253,0.145664
3,0500000US04013,"Maricopa County, Arizona",0.121441,0.239276,0.201831,0.122930
4,0500000US04015,"Mohave County, Arizona",0.136021,0.347913,0.081868,0.048427


In [20]:
cols_race = [3,5,7,8,9,11,12,13,14,15,16,17,18,19,20,21]
acs_race = acs_race.drop(acs_race.columns[cols_race], axis = 1)
acs_race.head()

,id,Geographic Area Name,Estimate!!Total:,Estimate!!Total:!!White alone,Estimate!!Total:!!Black or African American alone,Estimate!!Total:!!Asian alone
0,0500000US04001,"Apache County, Arizona",71887.0,15109.0,721.0,160.0
1,0500000US04003,"Cochise County, Arizona",125922.0,108177.0,6163.0,2795.0
2,0500000US04005,"Coconino County, Arizona",143476.0,91649.0,1364.0,2730.0
3,0500000US04013,"Maricopa County, Arizona",4485414.0,3547155.0,266128.0,192301.0
4,0500000US04015,"Mohave County, Arizona",212181.0,188756.0,1496.0,1923.0


In [21]:
#Converting them to percentages 

acs_race['White (Not Latino) Population'] = acs_race['Estimate!!Total:!!White alone']/acs_race['Estimate!!Total:'] 
acs_race['African American Population'] = acs_race['Estimate!!Total:!!Black or African American alone']/acs_race['Estimate!!Total:']
acs_race['Asian American Population'] = acs_race['Estimate!!Total:!!Asian alone']/acs_race['Estimate!!Total:']

acs_race.head()

,id,Geographic Area Name,Estimate!!Total:,Estimate!!Total:!!White alone,Estimate!!Total:!!Black or African American alone,Estimate!!Total:!!Asian alone,White (Not Latino) Population,African American Population,Asian American Population
0,0500000US04001,"Apache County, Arizona",71887.0,15109.0,721.0,160.0,0.210177,0.010030,0.002226
1,0500000US04003,"Cochise County, Arizona",125922.0,108177.0,6163.0,2795.0,0.859079,0.048943,0.022196
2,0500000US04005,"Coconino County, Arizona",143476.0,91649.0,1364.0,2730.0,0.638776,0.009507,0.019028
3,0500000US04013,"Maricopa County, Arizona",4485414.0,3547155.0,266128.0,192301.0,0.790820,0.059332,0.042873
4,0500000US04015,"Mohave County, Arizona",212181.0,188756.0,1496.0,1923.0,0.889599,0.007051,0.009063


In [22]:
# Dropping other columns we don't need now
acs_race = acs_race.drop(acs_race.columns[[3,4,5]], axis = 1)
acs_race.head()

,id,Geographic Area Name,Estimate!!Total:,White (Not Latino) Population,African American Population,Asian American Population
0,0500000US04001,"Apache County, Arizona",71887.0,0.210177,0.010030,0.002226
1,0500000US04003,"Cochise County, Arizona",125922.0,0.859079,0.048943,0.022196
2,0500000US04005,"Coconino County, Arizona",143476.0,0.638776,0.009507,0.019028
3,0500000US04013,"Maricopa County, Arizona",4485414.0,0.790820,0.059332,0.042873
4,0500000US04015,"Mohave County, Arizona",212181.0,0.889599,0.007051,0.009063


In [23]:
#Merge these two

#df2020 = pd.merge(left = acs_edu, right = acs_race, on = 'id')
#df2020.head()

In [24]:
# Drop repeated column from joining 
# df2020 = df2020.drop(['Geographic Area Name_y'],axis = 1)

In [25]:
# Reading in unemployment data
unemployment = pd.read_csv('data/unemployment2020.csv', header = 4)
unemployment.head()

,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CN0100100000000,1.0,1.0,"Autauga County, AL",Sep-19,"26,010","25,391",619,2.4
2,CN0100300000000,1.0,3.0,"Baldwin County, AL",Sep-19,"96,754","94,510","2,244",2.3
3,CN0100500000000,1.0,5.0,"Barbour County, AL",Sep-19,"8,656","8,376",280,3.2
4,CN0100700000000,1.0,7.0,"Bibb County, AL",Sep-19,"8,655","8,430",225,2.6


In [26]:
unemployment.tail(20)

,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
45050,CN7212100000000,72.0,121.0,"Sabana Grande Municipio, PR",Oct-20 p,"5,624","5,151",473,8.4
45051,CN7212300000000,72.0,123.0,"Salinas Municipio, PR",Oct-20 p,"6,431","5,745",686,10.7
45052,CN7212500000000,72.0,125.0,"San German Municipio, PR",Oct-20 p,"9,034","8,276",758,8.4
45053,CN7212700000000,72.0,127.0,"San Juan Municipio, PR",Oct-20 p,"128,360","117,918","10,442",8.1
45054,CN7212900000000,72.0,129.0,"San Lorenzo Municipio, PR",Oct-20 p,"10,771","9,910",861,8.0
45055,CN7213100000000,72.0,131.0,"San Sebastian Municipio, PR",Oct-20 p,"9,510","8,539",971,10.2
45056,CN7213300000000,72.0,133.0,"Santa Isabel Municipio, PR",Oct-20 p,"8,208","7,593",615,7.5
45057,CN7213500000000,72.0,135.0,"Toa Alta Municipio, PR",Oct-20 p,"25,611","23,731","1,880",7.3
45058,CN7213700000000,72.0,137.0,"Toa Baja Municipio, PR",Oct-20 p,"27,770","25,551","2,219",8.0
45059,CN7213900000000,72.0,139.0,"Trujillo Alto Municipio, PR",Oct-20 p,"25,534","23,915","1,619",6.3


In [27]:
unemployment.Period.unique()

array([nan, 'Sep-19', 'Oct-19', 'Nov-19', 'Dec-19', 'Jan-20', 'Feb-20',
       'Mar-20', 'Apr-20', 'May-20', 'Jun-20', 'Jul-20', 'Aug-20',
       'Sep-20', 'Oct-20 p'], dtype=object)

In [28]:
unemployment.dtypes

LAUS Code                          object
Code                              float64
Code.1                            float64
County Name/State Abbreviation     object
Period                             object
Force                              object
Employed                           object
Unemployed                         object
(%)                                object
dtype: object

In [29]:
# Converting code and code.1 to ints
unemployment = unemployment.fillna(0)
unemployment.Code = unemployment.Code.astype(int)
unemployment['Code.1'] = unemployment['Code.1'].astype(int)

unemployment.dtypes

LAUS Code                         object
Code                               int64
Code.1                             int64
County Name/State Abbreviation    object
Period                            object
Force                             object
Employed                          object
Unemployed                        object
(%)                               object
dtype: object

In [30]:
# Getting the data from Oct 2020
unemployment = unemployment[unemployment.Period == 'Oct-20 p']
unemployment = unemployment.reset_index()
unemployment.head()

,index,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
0,41848,CN0100100000000,1,1,"Autauga County, AL",Oct-20 p,"25,226","24,095","1,131",4.5
1,41849,CN0100300000000,1,3,"Baldwin County, AL",Oct-20 p,"95,660","90,920","4,740",5.0
2,41850,CN0100500000000,1,5,"Barbour County, AL",Oct-20 p,"9,634","8,880",754,7.8
3,41851,CN0100700000000,1,7,"Bibb County, AL",Oct-20 p,"8,624","8,127",497,5.8
4,41852,CN0100900000000,1,9,"Blount County, AL",Oct-20 p,"24,687","23,860",827,3.3


In [31]:
test = unemployment[unemployment['County Name/State Abbreviation'] == 'Presque Isle County, MI']
test.head()

,index,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
1300,43148,CN2614100000000,26,141,"Presque Isle County, MI",Oct-20 p,"4,829","4,602",227,4.7


In [32]:
# Creating a Fips column
unemployment['Fips'] = 0
for i in range(0,len(unemployment)):

    if(unemployment['Code'][i]) < 10:
        if (unemployment['Code.1'][i]) < 10:
            unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '00' + str(unemployment['Code.1'][i])
        elif (unemployment['Code.1'][i]) < 100:
            unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '0' + str(unemployment['Code.1'][i])
        else:
            unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '' + str(unemployment['Code.1'][i])
    elif (unemployment['Code.1'][i]) < 10:
        unemployment['Fips'][i] = str(unemployment['Code'][i]) + '00' + str(unemployment['Code.1'][i])
    elif (unemployment['Code.1'][i]) < 100:
        unemployment['Fips'][i] = str(unemployment['Code'][i]) + '0' + str(unemployment['Code.1'][i])
    else:
        unemployment['Fips'][i] = str(unemployment['Code'][i]) + '' + str(unemployment['Code.1'][i])
        
unemployment.head()

<ipython-input-32-dbd994a91222>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '00' + str(unemployment['Code.1'][i])
<ipython-input-32-dbd994a91222>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '0' + str(unemployment['Code.1'][i])
<ipython-input-32-dbd994a91222>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,index,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips
0,41848,CN0100100000000,1,1,"Autauga County, AL",Oct-20 p,"25,226","24,095","1,131",4.5,1001
1,41849,CN0100300000000,1,3,"Baldwin County, AL",Oct-20 p,"95,660","90,920","4,740",5.0,1003
2,41850,CN0100500000000,1,5,"Barbour County, AL",Oct-20 p,"9,634","8,880",754,7.8,1005
3,41851,CN0100700000000,1,7,"Bibb County, AL",Oct-20 p,"8,624","8,127",497,5.8,1007
4,41852,CN0100900000000,1,9,"Blount County, AL",Oct-20 p,"24,687","23,860",827,3.3,1009


In [33]:
# Dropping columns we don't need
unemployment = unemployment.drop(['LAUS Code','Code','Code.1', 'index'], axis = 1)
unemployment.head()

,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips
0,"Autauga County, AL",Oct-20 p,"25,226","24,095","1,131",4.5,1001
1,"Baldwin County, AL",Oct-20 p,"95,660","90,920","4,740",5.0,1003
2,"Barbour County, AL",Oct-20 p,"9,634","8,880",754,7.8,1005
3,"Bibb County, AL",Oct-20 p,"8,624","8,127",497,5.8,1007
4,"Blount County, AL",Oct-20 p,"24,687","23,860",827,3.3,1009


In [34]:
unemployment

,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips
0,"Autauga County, AL",Oct-20 p,"25,226","24,095","1,131",4.5,1001
1,"Baldwin County, AL",Oct-20 p,"95,660","90,920","4,740",5.0,1003
2,"Barbour County, AL",Oct-20 p,"9,634","8,880",754,7.8,1005
3,"Bibb County, AL",Oct-20 p,"8,624","8,127",497,5.8,1007
4,"Blount County, AL",Oct-20 p,"24,687","23,860",827,3.3,1009
...,...,...,...,...,...,...,...
3214,"Vega Baja Municipio, PR",Oct-20 p,"12,543","11,146","1,397",11.1,72145
3215,"Vieques Municipio, PR",Oct-20 p,"2,386","2,133",253,10.6,72147
3216,"Villalba Municipio, PR",Oct-20 p,"6,603","5,969",634,9.6,72149
3217,"Yabucoa Municipio, PR",Oct-20 p,"7,961","7,168",793,10.0,72151


In [35]:
len(unemployment)

3219

In [36]:
# Subsetting by states we needed
counter = 0
for i in range(1,len(unemployment)):
    if(('AZ' in unemployment['County Name/State Abbreviation'][i]) or 
      ('FL' in unemployment['County Name/State Abbreviation'][i]) or 
    ('GA' in unemployment['County Name/State Abbreviation'][i]) or 
      ('NV' in unemployment['County Name/State Abbreviation'][i]) or 
      ('PA' in unemployment['County Name/State Abbreviation'][i]) or 
      ('TX' in unemployment['County Name/State Abbreviation'][i]) or 
      ('IA' in unemployment['County Name/State Abbreviation'][i]) or 
      ('NC' in unemployment['County Name/State Abbreviation'][i]) or 
      ( 'WI' in unemployment['County Name/State Abbreviation'][i]) or 
      ('MI' in unemployment['County Name/State Abbreviation'][i]) or 
      ('OH' in unemployment['County Name/State Abbreviation'][i]) or 
      ('MI' in unemployment['County Name/State Abbreviation'][i])):
        counter+=1
    else:
        unemployment = unemployment.drop([i])
        
unemployment.head()
    

,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips
0,"Autauga County, AL",Oct-20 p,"25,226","24,095","1,131",4.5,1001
96,"Apache County, AZ",Oct-20 p,"20,064","17,523","2,541",12.7,4001
97,"Cochise County, AZ",Oct-20 p,"52,183","48,756","3,427",6.6,4003
98,"Coconino County, AZ",Oct-20 p,"70,110","63,315","6,795",9.7,4005
99,"Gila County, AZ",Oct-20 p,"20,992","19,379","1,613",7.7,4007


In [37]:
unemployment

,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips
0,"Autauga County, AL",Oct-20 p,"25,226","24,095","1,131",4.5,1001
96,"Apache County, AZ",Oct-20 p,"20,064","17,523","2,541",12.7,4001
97,"Cochise County, AZ",Oct-20 p,"52,183","48,756","3,427",6.6,4003
98,"Coconino County, AZ",Oct-20 p,"70,110","63,315","6,795",9.7,4005
99,"Gila County, AZ",Oct-20 p,"20,992","19,379","1,613",7.7,4007
...,...,...,...,...,...,...,...
3113,"Waukesha County, WI",Oct-20 p,"226,010","215,748","10,262",4.5,55133
3114,"Waupaca County, WI",Oct-20 p,"25,740","24,636","1,104",4.3,55135
3115,"Waushara County, WI",Oct-20 p,"11,285","10,788",497,4.4,55137
3116,"Winnebago County, WI",Oct-20 p,"92,735","89,021","3,714",4.0,55139


In [38]:
poverty = pd.read_csv('data/acs_poverty.csv', header = 1)

In [39]:
poverty.head()

,id,Geographic Area Name,Estimate!!Total!!Population for whom poverty status is determined,Margin of Error!!Total!!Population for whom poverty status is determined,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Under 5 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Under 5 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!5 to 17 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!5 to 17 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Related children of householder under 18 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Related children of householder under 18 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years!!18 to 34 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years!!18 to 34 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years!!35 to 64 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years!!35 to 64 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!60 years and over,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!60 years and over,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!65 years and over,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!65 years and over,Estimate!!Total!!Population for whom poverty status is determined!!SEX!!Male,Margin of Error!!Total!!Population for whom poverty status is determined!!SEX!!Male,Estimate!!Total!!Population for whom poverty status is determined!!SEX!!Female,Margin of Error!!Total!!Population for whom poverty status is determined!!SEX!!Female,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Black or African American alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Black or African American alone,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!American Indian and Alaska Native alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!American Indian and Alaska Native alone,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Asian alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Asian alone,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Estimate!!Total!!Population for whom 

In [40]:
# Getting just the columns that we need and renaming

poverty = poverty[['id','Geographic Area Name','Estimate!!Percent below poverty level!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED']]
poverty = poverty.rename(columns = {'Estimate!!Percent below poverty level!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED':'Poverty Percent, All Ages'})
poverty.head()

,id,Geographic Area Name,"Poverty Percent, All Ages"
0,0500000US04001,"Apache County, Arizona",42.8
1,0500000US04003,"Cochise County, Arizona",27.6
2,0500000US04005,"Coconino County, Arizona",38.0
3,0500000US04013,"Maricopa County, Arizona",21.0
4,0500000US04015,"Mohave County, Arizona",29.3


In [41]:
#Merging poverty into our 2020 data set
#df2020 = pd.merge(left = df2020, right = poverty, on = 'id')
#df2020.head()

In [42]:
unemployment.dtypes

County Name/State Abbreviation    object
Period                            object
Force                             object
Employed                          object
Unemployed                        object
(%)                               object
Fips                               int64
dtype: object

In [43]:
# Merge in our unemployment data
# make the id column of our df2020 match the fips column we created in
# the unemployment table
unemployment['Fips'] = unemployment['Fips'].apply(str)
#df2020["id"] = df2020["id"].str.replace("0500000US0", "")
#df2020 = pd.merge(left = df2020, right = unemployment, left_on = 'id', right_on = 'Fips', how = 'right')
#df2020.head()

In [44]:
#df2020

In [45]:
# Reading in 2020 election data
election = pd.read_csv('data/county_statistics.csv')

election.head()

,Unnamed: 0,county,state,percentage16_Donald_Trump,percentage16_Hillary_Clinton,total_votes16,votes16_Donald_Trump,votes16_Hillary_Clinton,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,lat,long,cases,deaths,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,0,Abbeville,SC,0.629,0.346,10724.0,6742.0,3712.0,0.661,0.330,12433.0,8215.0,4101.0,34.223334,-82.461707,805.0,17.0,24788.0,12044.0,12744.0,1.3,68.9,27.6,0.1,0.3,0.0,19452.0,35254.0,2259.0,19234.0,799.0,22.7,32.1,27.2,20.7,20.8,10.6,20.7,78.3,11.1,0.5,1.8,1.8,6.5,25.8,9505.0,78.8,13.3,7.8,0.1,9.4
1,1,Acadia,LA,0.773,0.206,27386.0,21159.0,5638.0,0.795,0.191,28425.0,22596.0,5443.0,30.295065,-92.414197,3182.0,102.0,62607.0,30433.0,32174.0,2.4,77.5,17.6,0.1,0.1,0.0,45197.0,40492.0,2544.0,21591.0,1002.0,21.5,27.6,27.6,16.9,25.7,15.0,14.8,83.2,10.3,0.2,1.6,2.2,2.5,27.6,24982.0,80.0,12.1,7.6,0.3,8.9
2,2,Accomack,VA,0.545,0.428,15755.0,8582.0,6737.0,0.542,0.447,16938.0,9172.0,7578.0,37.767072,-75.632346,1227.0,19.0,32840.0,16079.0,16761.0,8.8,60.3,28.3,0.3,0.7,0.0,24408.0,42260.0,2253.0,24266.0,1564.0,19.8,31.8,31.1,17.7,18.8,15.1,17.3,80.0,10.6,0.5,2.6,1.8,4.5,22.0,13837.0,74.6,18.1,7.1,0.2,5.4
3,3,Ada,ID,0.479,0.387,195587.0,93748.0,75676.0,0.504,0.465,259389.0,130699.0,120539.0,43.452658,-116.241552,17451.0,181.0,435117.0,217999.0,217118.0,7.9,85.2,1.2,0.4,2.6,0.1,316189.0,60151.0,1294.0,31642.0,725.0,11.8,13.1,43.0,16.6,25.0,6.9,8.4,80.7,7.7,0.5,1.5,2.8,6.9,20.4,214984.0,78.3,15.0,6.6,0.1,4.3
4,4,Adair,IA,0.653,0.300,3759.0,2456.0,1127.0,0.697,0.286,4183.0,2917.0,1197.0,41.330756,-94.471059,222.0,1.0,7192.0,3552.0,3640.0,1.7,96.6,0.3,0.0,0.4,0.0,5572.0,49477.0,2633.0,28861.0,2055.0,9.5,12.1,28.2,16.9,20.0,17.3,17.6,77.9,12.4,0.3,2.8,0.4,6.2,22.3,3680.0,73.8,15.3,10.4,0.5,3.0


In [46]:
cols = [0,3,4,5,6,7,13,14]

election = election.drop(election.columns[cols],axis = 1)
election.head()

,county,state,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,Abbeville,SC,0.661,0.330,12433.0,8215.0,4101.0,805.0,17.0,24788.0,12044.0,12744.0,1.3,68.9,27.6,0.1,0.3,0.0,19452.0,35254.0,2259.0,19234.0,799.0,22.7,32.1,27.2,20.7,20.8,10.6,20.7,78.3,11.1,0.5,1.8,1.8,6.5,25.8,9505.0,78.8,13.3,7.8,0.1,9.4
1,Acadia,LA,0.795,0.191,28425.0,22596.0,5443.0,3182.0,102.0,62607.0,30433.0,32174.0,2.4,77.5,17.6,0.1,0.1,0.0,45197.0,40492.0,2544.0,21591.0,1002.0,21.5,27.6,27.6,16.9,25.7,15.0,14.8,83.2,10.3,0.2,1.6,2.2,2.5,27.6,24982.0,80.0,12.1,7.6,0.3,8.9
2,Accomack,VA,0.542,0.447,16938.0,9172.0,7578.0,1227.0,19.0,32840.0,16079.0,16761.0,8.8,60.3,28.3,0.3,0.7,0.0,24408.0,42260.0,2253.0,24266.0,1564.0,19.8,31.8,31.1,17.7,18.8,15.1,17.3,80.0,10.6,0.5,2.6,1.8,4.5,22.0,13837.0,74.6,18.1,7.1,0.2,5.4
3,Ada,ID,0.504,0.465,259389.0,130699.0,120539.0,17451.0,181.0,435117.0,217999.0,217118.0,7.9,85.2,1.2,0.4,2.6,0.1,316189.0,60151.0,1294.0,31642.0,725.0,11.8,13.1,43.0,16.6,25.0,6.9,8.4,80.7,7.7,0.5,1.5,2.8,6.9,20.4,214984.0,78.3,15.0,6.6,0.1,4.3
4,Adair,IA,0.697,0.286,4183.0,2917.0,1197.0,222.0,1.0,7192.0,3552.0,3640.0,1.7,96.6,0.3,0.0,0.4,0.0,5572.0,49477.0,2633.0,28861.0,2055.0,9.5,12.1,28.2,16.9,20.0,17.3,17.6,77.9,12.4,0.3,2.8,0.4,6.2,22.3,3680.0,73.8,15.3,10.4,0.5,3.0


In [47]:
#cols2 = [range(9,43)]
#election = election.drop(election.columns[cols2],axis = 1)
#election.head()

In [48]:
temp = election[election.state == 'GA']
temp.head()

,county,state,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
72,Appling,GA,0.782,0.213,8341.0,6526.0,1779.0,1179.0,38.0,18471.0,9090.0,9381.0,9.7,69.6,17.9,0.6,0.8,0.0,13387.0,37089.0,3056.0,19936.0,1548.0,24.7,31.8,26.0,13.9,18.7,22.8,18.6,82.9,11.3,0.2,2.4,0.6,2.6,22.0,6951.0,78.8,16.2,4.9,0.2,7.7
97,Atkinson,GA,0.729,0.261,3155.0,2300.0,825.0,486.0,6.0,8313.0,4112.0,4201.0,24.6,56.6,17.8,0.7,0.0,0.0,5245.0,33063.0,1483.0,19904.0,3123.0,27.4,45.6,21.6,9.7,22.2,23.0,23.5,82.7,11.6,0.0,2.2,2.2,1.2,22.9,3289.0,82.8,11.6,5.6,0.0,8.4
111,Bacon,GA,0.861,0.134,4668.0,4018.0,625.0,640.0,16.0,11279.0,5599.0,5680.0,8.5,73.6,14.6,0.0,0.9,0.2,7903.0,38824.0,3793.0,18856.0,1372.0,23.2,34.4,30.7,12.0,22.4,18.6,16.2,79.1,15.1,1.2,1.3,1.3,2.1,20.4,4345.0,77.7,16.1,6.2,0.0,3.1
114,Baker,GA,0.577,0.419,1555.0,897.0,652.0,92.0,6.0,3251.0,1547.0,1704.0,5.4,52.1,41.3,0.2,0.6,0.5,2512.0,43867.0,6318.0,22270.0,2612.0,19.5,19.4,28.5,11.4,27.8,19.9,12.4,88.8,8.0,0.0,1.1,0.6,1.5,27.4,1286.0,70.1,23.1,6.8,0.0,2.5
117,Baldwin,GA,0.488,0.501,18251.0,8903.0,9140.0,2337.0,65.0,45527.0,22893.0,22634.0,2.2,52.9,41.4,0.3,1.5,0.2,36104.0,37008.0,2546.0,20114.0,1154.0,27.8,37.0,30.3,26.9,20.2,9.8,12.8,85.2,8.8,0.4,2.5,1.4,1.7,21.7,17647.0,68.6,27.7,3.7,0.0,9.0


In [49]:
election['county_state'] = election['county']+' County, '+election['state']
election.head()

,county,state,percentage20_Donald_Trump,percentage20_Joe_Biden,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,county_state
0,Abbeville,SC,0.661,0.330,12433.0,8215.0,4101.0,805.0,17.0,24788.0,12044.0,12744.0,1.3,68.9,27.6,0.1,0.3,0.0,19452.0,35254.0,2259.0,19234.0,799.0,22.7,32.1,27.2,20.7,20.8,10.6,20.7,78.3,11.1,0.5,1.8,1.8,6.5,25.8,9505.0,78.8,13.3,7.8,0.1,9.4,"Abbeville County, SC"
1,Acadia,LA,0.795,0.191,28425.0,22596.0,5443.0,3182.0,102.0,62607.0,30433.0,32174.0,2.4,77.5,17.6,0.1,0.1,0.0,45197.0,40492.0,2544.0,21591.0,1002.0,21.5,27.6,27.6,16.9,25.7,15.0,14.8,83.2,10.3,0.2,1.6,2.2,2.5,27.6,24982.0,80.0,12.1,7.6,0.3,8.9,"Acadia County, LA"
2,Accomack,VA,0.542,0.447,16938.0,9172.0,7578.0,1227.0,19.0,32840.0,16079.0,16761.0,8.8,60.3,28.3,0.3,0.7,0.0,24408.0,42260.0,2253.0,24266.0,1564.0,19.8,31.8,31.1,17.7,18.8,15.1,17.3,80.0,10.6,0.5,2.6,1.8,4.5,22.0,13837.0,74.6,18.1,7.1,0.2,5.4,"Accomack County, VA"
3,Ada,ID,0.504,0.465,259389.0,130699.0,120539.0,17451.0,181.0,435117.0,217999.0,217118.0,7.9,85.2,1.2,0.4,2.6,0.1,316189.0,60151.0,1294.0,31642.0,725.0,11.8,13.1,43.0,16.6,25.0,6.9,8.4,80.7,7.7,0.5,1.5,2.8,6.9,20.4,214984.0,78.3,15.0,6.6,0.1,4.3,"Ada County, ID"
4,Adair,IA,0.697,0.286,4183.0,2917.0,1197.0,222.0,1.0,7192.0,3552.0,3640.0,1.7,96.6,0.3,0.0,0.4,0.0,5572.0,49477.0,2633.0,28861.0,2055.0,9.5,12.1,28.2,16.9,20.0,17.3,17.6,77.9,12.4,0.3,2.8,0.4,6.2,22.3,3680.0,73.8,15.3,10.4,0.5,3.0,"Adair County, IA"


In [ ]:
# one model with 341 counties with all the data, we'd have to retrain 2016 using just those counties
# one model dropping the education, using the race data from the kaggle set, we'd have to retrain 2016
# without education
# one model with the 2018 data subbed for 2020
# 

In [ ]:
# look for education and race data and poverty, 

In [ ]:
# find election data
# GDP data comes out dec 9
# run one/a few of our 2016 models on the 2020 data as the 'test set'
# look at bias/variance decomp
# analyze mail in voting somehow
# build a 202 model including covid data?
# 